In [6]:
import yfinance as yf
import pandas as pd
import datetime as dt
import os

def pull_stock_data(tickers:str, start:str, end:str, interval:str, stripdateindex:str, column:str) -> pd.DataFrame:
    """
    Returns a dataframe with chosen stock information for last day of the month.

    Parameters:
    -------------
    tickers (str): Yahoo finance tickers for companies divided ONLY by single space
    start (str): start date of the period (format yyyy-mm-dd)
    end (str): end date of the period (format yyyy-mm-dd)
    interval (str): wanted interval (1d, 1m, 1y)
    stripdateindex (str): argument for stripping datetime index down (d - day, m - month, y- year)
    columns (list): list of wanted values, args same as in yahoo finance
    """
    # data download
    df = yf.download(tickers = tickers, start = start, end = end, interval = interval, groupby = 'ticker')
    # changing index from datetime to just year and month
    df['Date'] = df.index
    dfg = df.groupby([df.index.year, df.index.month], as_index=False).last()
    dfg.reset_index(inplace=True, drop=True)
    dfg.set_index('Date', inplace=True)
    dfg.index = pd.to_datetime(dfg.index).to_period(stripdateindex)
    # dropping na rows
    dfg.dropna(inplace = True)
    return dfg[column]
    
def save_to_desktop(dataframe:pd.DataFrame, file_name:str):
    """
    Saves dataframe to desktop in csv format under filename provided

    Parameters:
    ------------
    dataframe (pd.DataFrame): dataframe that is to be saved
    file_name (str): name of the file
    """
    # defining path to desktop on running unit
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    # saving file under chosen name on desktop
    dataframe.to_csv(desktop+'/'+file_name+'.csv', sep=';', encoding='UTF-8')



In [7]:
tick = "NKE MSFT XOM INTC CAT WMT JPM F UPS MKC"
st = '2000-12-01'
en = '2020-12-31'
intv = '1d'
strpdt = 'm'
col = ['Close', 'Volume']

dftest = pull_stock_data(tick, st, en, intv, strpdt, col)
dftest.head(20)

[*********************100%***********************]  11 of 11 completed


C:\Users\maxim\AppData\Local\Temp\ipykernel_14724\1805797520.py:22: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.index = pd.to_datetime(df.index).to_period(stripdateindex)


Adj Close                                                        \
               CAT          F       INTC        JPM       MKC       MSFT   
Date                                                                       
2000-12  13.426954  12.510590  17.972366  23.754143  5.809334  13.602913   
2001-01  12.549323  15.047402  22.119844  28.748064  5.904675  19.149916   
2001-02  11.895669  15.004977  17.075615  24.571960  6.366357  18.503101   
2001-03  12.690614  15.172235  15.739306  23.645111  6.802119  17.150644   
2001-04  14.354870  15.906034  18.489395  25.267097  6.397518  21.247196   
2001-05  15.596690  13.272593  16.156540  26.096527  6.576585  21.695667   
2001-06  14.413116  13.381607  17.507395  23.611650  6.840298  22.893667   
2001-07  15.867390  13.883074  17.842579  22.990427  6.982382  20.757959   
2001-08  14.493451  10.830654  16.735270  21.076153  7.392922  17.891554   
2001-09  12.986133   9.569782  12.241856  18.267790  7.491058  16.047514   
2001-10  12.962943   8.852737  14.625537  18.915035  7.191115  18.236517   
2001-11  13.848892  10.545334  19.560606  20.375471  7.066223  20.137022   
2001-12  15.259482   8.752521  18.850256  19.635422  6.896966  20.776779   
2002-01  14.684151   8.518673  21.002001  18.393015  7.299296  19.980213   
2002-02  16.330578   8.341926  17.112074  15.949353  8.091979  18.296114   
2002-03  16.724802   9.244514  18.237642  19.439117  8.443733  18.913927   
2002-04  16.068754   8.969810  17.158146  19.139221  8.504745  16.389349   
2002-05  15.474460   9.894824  16.564407  19.793303  8.710398  15.965972   
2002-06  14.491583   9.026223  10.964849  18.675625  8.541234  17.154572   
2002-07  13.233379   7.598953  11.276931  13.742438  7.577254  15.047092   

                                                    ...       Volume  \
              NKE        UPS        WMT        XOM  ...            F   
Date                                                ...                
2000-12  5.512078  33.709515  35.416012  21.869669  ...  116791300.0   
2001-01  5.440065  35.516918  37.913372  21.168455  ...  126191800.0   
2001-02  3.859770  32.435726  33.434525  20.388624  ...   90557300.0   
2001-03  3.990307  32.753685  33.708187  20.482918  ...  131030200.0   
2001-04  4.141493  33.070267  34.586437  22.404755  ...  113711700.0   
2001-05  4.071163  34.221554  34.593140  22.442692  ...  145986800.0   
2001-06  4.159323  33.378872  32.621155  22.197529  ...  118509600.0   
2001-07  4.716828  32.818714  37.420879  21.229017  ...   84425600.0   
2001-08  4.959862  31.894722  32.165890  20.410551  ...  131204300.0   
2001-09  4.643422  30.120956  33.136551  20.141600  ...  129333500.0   
2001-10  4.902421  29.553059  34.461643  20.167160  ...  126336100.0   
2001-11  5.262952  32.577904  36.975880  19.119183  ...  110822200.0   
2001-12  5.585742  31.691906  38.584980  20.208231  ...  144339500.0   
2002-01  5.956802  33.424763  40.264725  20.079691  ...  344648200.0   
2002-02  5.852404  34.273769  41.627468  21.236641  ...  180655600.0   
2002-03  5.966746  35.474033  41.150841  22.671566  ...  203611100.0   
2002-04  5.307805  35.030624  37.543335  20.778395  ...  159584500.0   
2002-05  5.349605  35.228977  36.360432  20.654236  ...  181075200.0   
2002-06  5.339655  36.146221  36.972023  21.288706  ...  149817300.0   
2002-07  4.911560  38.247669  33.096115  19.124456  ...  235272400.0   

                                                                           \
                 INTC          JPM         MKC          MSFT          NKE   
Date                                                                        
2000-12  1.120692e+09  200159000.0   9706000.0  2.056668e+09  242557600.0   
2001-01  1.211435e+09  247102800.0  11913600.0  2.005531e+09  307727200.0   
2001-02  9.094086e+08  154547500.0  11599600.0  1.536896e+09  310156800.0   
2001-03  1.421495e+09  204257600.0  18193600.0  1.895350e+09  227849600.0   
2001-04  1.232602e+09  167154700.0  12639200.0  2.075807e+09  19

In [38]:
df1 = dftest['Close']
df1.head()

,CAT,F,INTC,JPM,MKC,MSFT,NKE,UPS,WMT,XOM,^IRX
Date,,,,,,,,,,,
2000-12,13.426954,12.510590,17.972366,23.754143,5.809334,13.602913,5.512078,33.709515,35.416012,21.869669,5.73
2001-01,12.549323,15.047402,22.119844,28.748064,5.904675,19.149916,5.440065,35.516918,37.913372,21.168455,4.84
2001-02,11.895669,15.004977,17.075615,24.571960,6.366357,18.503101,3.859770,32.435726,33.434525,20.388624,4.72
2001-03,12.690614,15.172235,15.739306,23.645111,6.802119,17.150644,3.990307,32.753685,33.708187,20.482918,4.18
2001-04,14.354870,15.906034,18.489395,25.267097,6.397518,21.247196,4.141493,33.070267,34.586437,22.404755,3.83


In [39]:
df1 = df1.pct_change()

In [10]:
save_to_desktop(dftest, 'data')

In [42]:
fama = pd.read_csv(r"F-F_Research_Data_Factors.CSV", sep=';')
fama['Date'] = fama['Date'].astype('string')

fama['Date']= pd.to_datetime(fama['Date'], format='%Y%m')
fama.set_index('Date', inplace=True)
fama.index = fama.index.to_period('m')
fama = fama.divide(100)
fama.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07,0.0296,-0.0256,-0.0243,0.0022
1926-08,0.0264,-0.0117,0.0382,0.0025
1926-09,0.0036,-0.0140,0.0013,0.0023
1926-10,-0.0324,-0.0009,0.0070,0.0032
1926-11,0.0253,-0.0010,-0.0051,0.0031


In [34]:

#fama.head()

In [47]:
dffull = pd.merge(df1,fama, how='left', left_index=True, right_index=True)
del dffull['^IRX']
dffull.head()

,CAT,F,INTC,JPM,MKC,MSFT,NKE,UPS,WMT,XOM,Mkt-RF,SMB,HML,RF
Date,,,,,,,,,,,,,,
2000-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0119,0.0072,0.0761,0.0050
2001-01,-0.065363,0.202773,0.230770,0.210234,0.016412,0.407781,-0.013064,0.053617,0.070515,-0.032063,0.0313,0.0669,-0.0509,0.0054
2001-02,-0.052087,-0.002819,-0.228041,-0.145266,0.078189,-0.033776,-0.290492,-0.086753,-0.118134,-0.036839,-0.1005,-0.0079,0.1248,0.0038
2001-03,0.066826,0.011147,-0.078258,-0.037720,0.068448,-0.073094,0.033820,0.009803,0.008185,0.004625,-0.0726,0.0024,0.0642,0.0042
2001-04,0.131141,0.048365,0.174727,0.068597,-0.059482,0.238857,0.037888,0.009666,0.026055,0.093826,0.0794,0.0055,-0.0468,0.0039


In [50]:

cols = dffull.columns.tolist()
# Calculate excess stock return for each company
for i in cols[:10]:
    dffull[i+'_re'] = dffull[i]-dffull['RF']
    # Calculate excess Market return
dffull['reM'] = dffull['Mkt-RF']-dffull['RF']
dffull.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 241 entries, 2000-12 to 2020-12
Freq: M
Data columns (total 25 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CAT      240 non-null    float64
 1   F        240 non-null    float64
 2   INTC     240 non-null    float64
 3   JPM      240 non-null    float64
 4   MKC      240 non-null    float64
 5   MSFT     240 non-null    float64
 6   NKE      240 non-null    float64
 7   UPS      240 non-null    float64
 8   WMT      240 non-null    float64
 9   XOM      240 non-null    float64
 10  Mkt-RF   241 non-null    float64
 11  SMB      241 non-null    float64
 12  HML      241 non-null    float64
 13  RF       241 non-null    float64
 14  CAT_re   240 non-null    float64
 15  F_re     240 non-null    float64
 16  INTC_re  240 non-null    float64
 17  JPM_re   240 non-null    float64
 18  MKC_re   240 non-null    float64
 19  MSFT_re  240 non-null    float64
 20  NKE_re   240 non-null    float64
 2

In [52]:
import statsmodels.api as sm

In [51]:
cols1 = dffull.columns.tolist()

colsreg = [x for x in cols1 if '_re' in x]

In [56]:
X = sm.add_constant(dffull['reM'])
for col in colsreg[:2]:
    y = dffull[col]
    # fajna regresja y na X tutaj
    


         const    CAT_re
Date                    
2000-12    1.0       NaN
2001-01    1.0 -0.070763
2001-02    1.0 -0.055887
2001-03    1.0  0.062626
2001-04    1.0  0.127241
         const      F_re
Date                    
2000-12    1.0       NaN
2001-01    1.0  0.197373
2001-02    1.0 -0.006619
2001-03    1.0  0.006947
2001-04    1.0  0.044465
